### IMPORTING LIBRARIES

In [1]:
import pandas as pd, numpy as np, matplotlib as mpl, math

import geopandas as gpd
from geopandas import GeoDataFrame
pd.options.display.float_format = '{:.2f}'.format
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

%matplotlib inline
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt

import seaborn as sns

#from scipy import ndimage
from scipy import stats
from scipy.stats import t, chisquare, iqr
#pandas.set_option('display.max_rows', None)
from scipy.stats import ttest_ind

import warnings 
warnings.filterwarnings('ignore')

In [2]:
pip install researchpy

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import researchpy as rp

In [4]:
# Filepath for csv == 'PropertyData/Property Data.csv'

# ANSWER KEY BELOW 

### CONNECTING TO DATA

In [5]:
raw = pd.read_csv('PropertyData/Property Data.csv')
raw.head()

,id,price_000,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,apt,floor,...,pcnt_indu,pcnt_com,pcnt_insti,pcnt_vacant,pcn_green,homicides,house,ses_bin,lnprice,price_hi
0,40003,60000,1,0,0,0,0,0,1,5,...,0.00,0.00,0.49,0.00,1.74,39.92,0,0,11.00,0
1,40007,140000,0,1,0,0,0,0,0,0,...,0.00,15.41,1.32,0.00,0.54,46.00,1,0,11.85,1
2,40008,38000,0,1,0,0,0,0,1,1,...,0.00,8.16,5.57,0.00,1.55,45.87,0,0,10.55,0
3,40010,110000,0,1,0,0,0,0,0,0,...,0.00,8.11,5.53,0.00,1.58,45.88,1,0,11.61,1
4,40011,120000,0,1,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.40,46.00,1,0,11.70,1


#### Data Cleaning

In [6]:
raw.dtypes

id               int64
price_000        int64
yr_2001          int64
yr_2002          int64
yr_2003          int64
yr_2004          int64
yr_2005          int64
yr_2006          int64
apt              int64
floor            int64
age_0_10         int64
age_10_20        int64
age_20_more      int64
num_room         int64
num_bath         int64
num_garage       int64
area_m2          int64
ses              int64
pop_dens       float64
pcnt_indu      float64
pcnt_com       float64
pcnt_insti     float64
pcnt_vacant    float64
pcn_green      float64
homicides      float64
house            int64
ses_bin          int64
lnprice        float64
price_hi         int64
dtype: object

In [7]:
# Convert Variable Type
raw['id'] = raw['id'].astype(str)
raw.dtypes

id              object
price_000        int64
yr_2001          int64
yr_2002          int64
yr_2003          int64
yr_2004          int64
yr_2005          int64
yr_2006          int64
apt              int64
floor            int64
age_0_10         int64
age_10_20        int64
age_20_more      int64
num_room         int64
num_bath         int64
num_garage       int64
area_m2          int64
ses              int64
pop_dens       float64
pcnt_indu      float64
pcnt_com       float64
pcnt_insti     float64
pcnt_vacant    float64
pcn_green      float64
homicides      float64
house            int64
ses_bin          int64
lnprice        float64
price_hi         int64
dtype: object

In [8]:
# Check for Null Values
raw.isnull().sum()

id             0
price_000      0
yr_2001        0
yr_2002        0
yr_2003        0
yr_2004        0
yr_2005        0
yr_2006        0
apt            0
floor          0
age_0_10       0
age_10_20      0
age_20_more    0
num_room       0
num_bath       0
num_garage     0
area_m2        0
ses            0
pop_dens       0
pcnt_indu      0
pcnt_com       0
pcnt_insti     0
pcnt_vacant    0
pcn_green      0
homicides      0
house          0
ses_bin        0
lnprice        0
price_hi       0
dtype: int64

#### Create Sub-Dataframe

In [9]:
# Create Sub-Dataframe
df = raw[['house','apt','price_000','age_0_10','age_20_more','pcn_green','num_room']].copy()
df.head()

,house,apt,price_000,age_0_10,age_20_more,pcn_green,num_room
0,0,1,60000,1,0,1.74,3
1,1,0,140000,0,0,0.54,4
2,0,1,38000,0,0,1.55,3
3,1,0,110000,0,0,1.58,4
4,1,0,120000,0,0,0.40,4


In [10]:
# Rename variables of interest
df.rename(columns={"price_000":"price", "age_0_10":"age_new", "age_20_more":"age_old", "num_room":"rooms"}, inplace = True)

#### Describing Variables

In [11]:
# Property Price -- Continous Variable

df['price'].describe()

count     3976.00
mean     93511.05
std      75516.07
min      20000.00
25%      50000.00
50%      72000.00
75%     110000.00
max     800000.00
Name: price, dtype: float64

In [12]:
# Next week we'll learn how to use a histogram to visualize the distribution of a continous variable
# If we're only interested in certain statistics, we can also call them up specifically 
# Note the median is equal to the 50% percentile above, and IQR is equal to the 75th percentile minus the 25th

print ("The Price Median is " + str(df['price'].median()))
print ("The Price IQR is " + str(iqr(df['price'])))

The Price Median is 72000.0
The Price IQR is 60000.0


In [13]:
# House -- Discrete Numeric Variable (Dummy Variable)

df['house'].describe()

count   3976.00
mean       0.32
std        0.47
min        0.00
25%        0.00
50%        0.00
75%        1.00
max        1.00
Name: house, dtype: float64

In [14]:
# We can also use the value_counts function (in general, it gives us a better sense of categorical variables)

# Value_Counts defaults to counts
df['house'].value_counts()

0    2690
1    1286
Name: house, dtype: int64

In [15]:
# And we can normalize value_counts to get percentages
df["house"].value_counts(normalize=True)

0   0.68
1   0.32
Name: house, dtype: float64

In [16]:
# If we want to see counts and percentages together, we can concatenate these outputs into one table

# Defining a function called 'tab' that can take in any dataframe and any variable and return the output below
# What steps could you take to apply this function to a different dataframe? 
### (hint: df in the function, and df at the bottom where we call it with 'house' aren't the same thing...)
def tab(df, x):
    print ("Total Count", df[x].count())
    print ("Total Pct", sum(df[x].value_counts(normalize=True)))
    
    return pd.concat([ df[x].value_counts(), df[x].value_counts(normalize=True) ], 
                     axis=1, keys=('counts','pct'))

tab(df, 'house')

Total Count 3976
Total Pct 1.0


,counts,pct
0,2690,0.68
1,1286,0.32


In [17]:
# These functions have been helpful for individual variables, but say you want to see summary stats for ALL the 
# variables in your dataframe at once?

df.describe().T

# (Try deleting the .T to see what happens if you don't use it - either way is fine!)

,count,mean,std,min,25%,50%,75%,max
house,3976.00,0.32,0.47,0.00,0.00,0.00,1.00,1.00
apt,3976.00,0.68,0.47,0.00,0.00,1.00,1.00,1.00
price,3976.00,93511.05,75516.07,20000.00,50000.00,72000.00,110000.00,800000.00
age_new,3976.00,0.17,0.37,0.00,0.00,0.00,0.00,1.00
age_old,3976.00,0.20,0.40,0.00,0.00,0.00,0.00,1.00
pcn_green,3976.00,3.37,7.87,0.00,0.32,0.98,3.37,57.00
rooms,3976.00,3.05,1.05,1.00,2.00,3.00,4.00,7.00


In [18]:
# If you want to see summary statistics like this for just a few variables, see the Appendix Code at the bottom!

### STATISTICAL TESTS

In [19]:
# Define universal set of statistics to be called with ".agg" command
stats = ['count','min','max','mean', 'median', 'std']

#### Test A. T-Test (of Means)

In [20]:
# Test a. Descriptive Statistics
# The groupby and aggregate functions are helpful for looking at crosstabulated summary statistics

df["price"].groupby(df["apt"]).agg(stats)

,count,min,max,mean,median,std
apt,,,,,,
0,1286,20000,740000,123234.51,105000,76437.63
1,2690,20000,800000,79301.24,60000,70805.11


In [21]:
# Test a. Do apartments have different prices than houses?

# Variables
apt_p = df[df.apt == 1].price #Apartment Price
n_apt_p = df[df.apt == 0].price #Non-Apartment Price

# T-Test
ttest_ind(apt_p, n_apt_p, equal_var = False, nan_policy="omit")

Ttest_indResult(statistic=-17.35662262818821, pvalue=1.2993761491374771e-63)

In [22]:
# If you wanted to normalize the price of a property by the number of bedrooms, how would you change the code? 
# (hint: more than one right answer!!)

# Create Per Room Price Variable within Dataframe
df['pp_rm'] = df['price']/df['rooms']

# Variables for T-Test
apt_rm_p = df[df.apt == 1].pp_rm #Apartment Price per Room
n_apt_rm_p = df[df.apt == 0].pp_rm #Non-Apartment Price per Room

# T-Test
ttest_ind(apt_rm_p, n_apt_rm_p, equal_var = False, nan_policy="omit")

Ttest_indResult(statistic=0.13576881903359253, pvalue=0.8920129308585947)

#### Test B. T-Test (of Means)

In [23]:
# Test b. Descriptive Statistics

df["pcn_green"].groupby(df["apt"]).agg(stats)

,count,min,max,mean,median,std
apt,,,,,,
0,1286,0.00,57.00,2.56,1.29,4.88
1,2690,0.00,57.00,3.76,0.93,8.93


In [24]:
# Test b. Are houses more likely to be located in neighborhoods with more greenspace than apartments?

# Variables
apt_gr = df[df.apt == 1].pcn_green #Apartment neighborhood's percent green space
n_apt_gr = df[df.apt == 0].pcn_green #Non-Apartment neighborhood's percent green space

# T-Test
ttest_ind(apt_gr, n_apt_gr, equal_var = False, nan_policy="omit")

Ttest_indResult(statistic=5.456139603111823, pvalue=5.1676797595980235e-08)

#### Test C. T-Test for Subset of Dataframe

In [25]:
# Test c. Descriptive Statistics
apt_p = (df[df.apt == 1].price) #Price only of apartments

#Group price of apartments by new vs. old
apt_p.groupby(df["age_new"]).agg(stats)

,count,min,max,mean,median,std
age_new,,,,,,
0,2082,20000,800000,81620.57,62000,75097.56
1,608,20000,500000,71359.08,58000,52847.82


In [26]:
# Test c. Is the price of newer apartments different from older apartments?

# Variables
o_apt_p = df[(df.age_new == 0) & (df.apt == 1)].price #Price of old apartments
y_apt_p = df[(df.age_new == 1) & (df.apt == 1)].price #Price of young apartments

# T-Test
ttest_ind(o_apt_p, y_apt_p, equal_var = False, nan_policy="omit")

Ttest_indResult(statistic=3.7973397283310604, pvalue=0.00015254710482132656)

#### Test D. Chi-Square Test (of Proportions)

In [27]:
# Test d. Descriptive Statistics

pd.crosstab(df['house'], df['age_old'], margins=True, margins_name='Total')

age_old,0,1,Total
house,,,
0,2381,309,2690
1,817,469,1286
Total,3198,778,3976


In [28]:
# Normalizing by row ('index') - could also normalize by 'columns'

pd.crosstab(df['house'], df['age_old'], normalize='index', margins=True, margins_name='Total')

age_old,0,1
house,,
0,0.89,0.11
1,0.64,0.36
Total,0.80,0.20


In [29]:
# Test d. Are houses (y/n) more likely to be older (age_20_more) or younger?

table,results = rp.crosstab(df["house"], df["age_old"], prop="row", test="chi-square")
table

age_old             
            0     1    All
house                     
0       88.51 11.49 100.00
1       63.53 36.47 100.00
All     80.43 19.57 100.00

In [30]:
results

,Chi-square test,results
0,Pearson Chi-square ( 1.0) =,345.03
1,p-value =,0.00
2,Cramer's phi =,0.29


### Appendix - Defining Functions

In [31]:
# Showing Null Values in Order
# Will only show null counts for rows that have more than 0 null values

### DEFINING THE FUNCTION
def var_nulls (df):
    null_counts = df.isnull().sum() #Sum of null counts attributed to a variable
    return null_counts[null_counts > 0].sort_values(ascending=False) #Sort values greater than 0 largest to smallest

var_nulls(raw)

#Output shows blank list because we have no null values here

Series([], dtype: int64)

In [32]:
# Value Counts - Seeing Counts and Percentages Together

### DEFINING THE FUNCTION
def tab(df, x):
    print ("Total Count", df[x].count())
    print ("Total Pct", sum(df[x].value_counts(normalize=True)))
    return pd.concat([df[x].value_counts(), df[x].value_counts(normalize=True)], 
                     axis=1, keys=('counts','pct'))

### CALL THE FUNCTION
tab(df, 'house') #Specify the dataframe and the variable in the parentheses

Total Count 3976
Total Pct 1.0


,counts,pct
0,2690,0.68
1,1286,0.32


In [33]:
# We used the groupby function before each statistical test we used - we can also call this up with one function!

### DEFINING THE FUNCTION
def grpby_stats (df, var1, var2):
    stat = ['count', 'mean', 'min', 'max', 'median', 'std'] #Specify the statistics we want
    return df[var1].groupby(df[var2]).agg(stat) #Group the first variable by the second variable, and aggregate stats

### CALL THE FUNCTION
grpby_stats(df, 'price', 'apt') #Specify the dataframe, key variabe, and grouping variable in the parentheses

#Does this work for Test C, where we define apt_p = (df[df.apt == 1].price)? Try it out!

,count,mean,min,max,median,std
apt,,,,,,
0,1286,123234.51,20000,740000,105000,76437.63
1,2690,79301.24,20000,800000,60000,70805.11


In [34]:
# If you want to see this for ALL the variables in your dataframe, you use this code.
# This asks Python to describe the entire dataframe, and transpose (T) the columns and rows
# (Try deleting the .T to see what happens if you don't use it - either way is fine!)

df.describe().T

,count,mean,std,min,25%,50%,75%,max
house,3976.00,0.32,0.47,0.00,0.00,0.00,1.00,1.00
apt,3976.00,0.68,0.47,0.00,0.00,1.00,1.00,1.00
price,3976.00,93511.05,75516.07,20000.00,50000.00,72000.00,110000.00,800000.00
age_new,3976.00,0.17,0.37,0.00,0.00,0.00,0.00,1.00
age_old,3976.00,0.20,0.40,0.00,0.00,0.00,0.00,1.00
pcn_green,3976.00,3.37,7.87,0.00,0.32,0.98,3.37,57.00
rooms,3976.00,3.05,1.05,1.00,2.00,3.00,4.00,7.00
pp_rm,3976.00,31914.10,24523.78,5000.00,17666.67,26000.00,37500.00,543000.00


In [35]:
# What if you just want to view a few of your key variables of interest?

# List of key variables
key_var = ['apt','price','rooms']

# Code calling a few variables 
df[key_var].describe().T

# You could also use this code below if you don't want to separately define a list of variables: 
# df[['apt','price','rooms']].describe().T

,count,mean,std,min,25%,50%,75%,max
apt,3976.00,0.68,0.47,0.00,0.00,1.00,1.00,1.00
price,3976.00,93511.05,75516.07,20000.00,50000.00,72000.00,110000.00,800000.00
rooms,3976.00,3.05,1.05,1.00,2.00,3.00,4.00,7.00
